# Lung Diseases Classification - Phase 2: Optimization Hyperparameters Search with Optuna

<div style="text-align: center;">
    <img src="../../images/optuna_logo.jpg" alt="Crop with image and mask" style="display: block; margin: 0 auto;">
</div>


## Context
This notebook represents the **second phase** of the lung disease classification pipeline, focusing on **optimization-level hyperparameter tuning** after the model architecture has already been fixed.

In the previous phase, Optuna was used to search for the **best classifier head architecture** on top of a frozen DenseNet121 backbone, including:
- Number of dense layers in the classification head  
- Number of units per dense layer  

With the architecture now finalized, this notebook shifts attention to **training dynamics and regularization**, which play a critical role in model stability, convergence behavior, and generalization performance—especially in medical imaging tasks.

---

## Objective

The main goal of this notebook is to identify the **optimal optimization hyperparameters** for a multi-class lung disease classifier that predicts:
- **COVID-19**
- **Viral Pneumonia**
- **Lung Opacity**

using ROI-focused chest X-ray images processed through a DenseNet121 backbone.

The optimization process is driven by **Optuna**, guided by a **custom penalized F1-score objective** that balances predictive performance with training stability.


## Key Optimization Parameters Explored

This phase focuses on tuning **training dynamics and regularization hyperparameters** that directly affect convergence behavior, stability, and generalization performance. The following parameters are explored using Optuna:

- **Stage-wise Learning Rates**
  - `lr_stage1`: Learning rate used during the initial frozen-backbone training stage  
  - `lr_stage2`: Lower learning rate applied during fine-tuning for stable convergence  

- **Weight Decay**
  - Controls L2 regularization strength to reduce overfitting and improve generalization  

- **Dropout Rate**
  - Applied to classifier head layers to regularize dense representations and prevent co-adaptation  

- **Label Smoothing**
  - Introduces controlled uncertainty in ground-truth labels to improve calibration and robustness  

- **Class Weight Power (`weight_power`)**
  - A tunable exponent applied to class weights  
  - `0.0` enforces equal weighting across classes  
  - `1.0` applies raw class imbalance weights  
  - Intermediate values allow smooth interpolation between these extremes  

All optimization parameters are evaluated jointly under a consistent training protocol, ensuring that improvements are driven by learning dynamics rather than architectural changes.


## Training & Evaluation Strategy

- **Backbone**: ImageNet-pretrained DenseNet121  
- **Input Focus**: Lung ROI cropping based on segmentation masks (no hard masking)  
- **Data Pipeline**:
  - TFRecord-based datasets
  - Class remapping and one-hot encoding
  - Optional augmentation applied only during training
- **Metrics**:
  - Accuracy
  - Precision
  - Recall
  - Macro F1-score
  - AUC

A **custom penalized F1-score** is used as the Optuna objective, discouraging:
- Large precision–recall gaps
- Overfitting (via train–validation loss divergence)

## Utilities & Modularity

To maintain clarity and reproducibility, all reusable logic is abstracted into a dedicated `utils.py` file, including:
- Dataset construction and preprocessing
- Model creation and compilation
- Reproducibility utilities
- Memory cleanup between trials
- Custom scoring functions

This design keeps the notebook focused on **experiment orchestration**, while core logic remains centralized and reusable.

## Outcome

At the end of this notebook, the Optuna study produces:
- The **best-performing optimization hyperparameters**
- A statistically grounded configuration ready for:
  - Final training
  - Cross-validation
  - Deployment or downstream disease-specific modeling stages

This phase completes the systematic two-stage Optuna search strategy:
1. **Architecture search**
2. **Optimization hyperparameter search**

forming a robust foundation for reliable lung disease classification.


## Section 1 - Environment Setup & Experiment Initialization

This section initializes the experimental environment required for the Optuna-based optimization process.  
It establishes consistent dependencies, enforces reproducibility, configures GPU memory behavior, and selects the most suitable hardware execution strategy.

These steps are essential to ensure that all optimization trials are conducted under **controlled, repeatable, and hardware-aware conditions**, allowing fair comparison across experiments.


### 1.1 Library Imports and Dependencies

Imports all core libraries required for configuration management, numerical computation, model training, and hyperparameter optimization.  
Reusable project-specific logic is centralized in a shared `utils.py` module, keeping the notebook focused on experiment orchestration rather than implementation details.


In [1]:
import json
import numpy as np
import optuna
import os
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
import time

# Import shared utilities for data pipelines, model construction,
# training helpers, and experiment management
from utils import *

2026-01-31 19:43:09.062228: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-31 19:43:09.611859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-31 19:43:12.893469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### 1.2 Framework Version Logging

Logging the TensorFlow/Keras version provides traceability and helps ensure consistency across experimental runs.  
This is particularly important when comparing results across environments or revisiting experiments at a later stage.


In [2]:
# Log TensorFlow/Keras version for experiment reproducibility and debugging
print(f"TensorFlow / Keras Version: {tf.__version__}")

TensorFlow / Keras Version: 2.20.0


### 1.3 Reproducibility and Hardware Configuration

Now enforce deterministic behavior by seeding all relevant random number generators.  
It also enables safe GPU memory growth to avoid unnecessary memory allocation and automatically selects the optimal distribution strategy (TPU, multi-GPU, or CPU) based on the available hardware.

In [3]:
# Global seed to ensure deterministic behavior across runs
SEED = 28
seed_everthing(SEED)

# Enable safe GPU memory allocation (prevents full memory reservation)
gpu_growth()

# Automatically select the optimal distribution strategy
# (TPU → Multi-GPU → CPU fallback)
strategy = get_strategy()

For reproducibility, everything seeded!
Enabled memory growth for 1 GPU(s)
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Using GPU strategy: MirroredStrategy
REPLICAS: 1


I0000 00:00:1768070208.911453   71042 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2246 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


## Section 2 – Global Configuration and Dataset Metadata

Ok in this Section define all **global constants and configuration dictionaries** used throughout the optimization process.  
Centralizing these parameters ensures **clarity**, **reproducibility**, and **consistent behavior** across Optuna trials.

The configuration is split into logical groups covering:
- Data pipeline behavior
- Model and optimization settings
- Training-scale parameters under distributed execution
- Dataset label mappings

### 2.1 Core System and Data Parameters

Now we define fundamental constants that control dataset loading, input dimensions, and model structure.  
These values remain fixed across all optimization trials, ensuring that Optuna explores **optimization dynamics only**, not structural variations.

Key parameters such as image resolution, number of classes, and shuffle buffer size directly influence both **training stability** and **data throughput**.

#### 2.1.1 Algorithm and Optimization Configuration

Then groups all algorithm-level configuration dictionaries used during training and optimization.  
Each dictionary encapsulates a specific aspect of the experimental protocol, making the setup both **explicit** and **easily adjustable**.

Notably:
- The penalized $F_1$ configuration controls how performance is evaluated across training stages.
- Optimization parameters define the number of Optuna trials and the fine-tuning schedule.
- Model configuration fixes the input and output interface of the network.

#### 2.1.2 Distributed Training Batch Configuration

This subsection defines batch-size scaling under distributed training.  
The **global batch size** is computed as:

$$
\text{Global Batch Size} = \text{Batch Size per Replica} \times N_{\text{replicas}}
$$

This formulation ensures that training behavior remains consistent when scaling across multiple devices.


In [4]:
# Enable automatic tuning of tf.data parallelism
AUTO = tf.data.AUTOTUNE
# Base directories for dataset files and saved models
DATA_DIR = '../../data/tfrecords/'
MODELS_DIR = '../../models/'
# Input image resolution used throughout training
IMAGE_SIZE = (256, 256)
# Buffer size for dataset shuffling
SHUFFLE_SIZE = 1024
# Number of target disease classes (COVID, Viral Pneumonia, Lung Opacity)
NUM_CLASSES = 3
# Mask resolution is kept identical to image resolution
MASK_SIZE = IMAGE_SIZE
# Backbone layer name used as the starting point for fine-tuning
UNFREEZE_LAYER = 'conv5_block1_0_bn'
# === ALGORITHM CONFIGS ===
# Configuration for the penalized F1-score objective
# alpha_p controls penalty strength, stage defines evaluation window
PENALIZED_F1_CONFIG = {'alpha_p': 0.5, 'stage': 3}
# Optuna-driven optimization schedule and fine-tuning control
OPTIMIZATION_CONFIG = {
    'opt_trials': 30,              # Number of Optuna trials
    'opt_warmup_epoch': 3,          # Frozen-backbone warm-up epochs
    'opt_unfreeze_epoch': 5,        # Epoch to start fine-tuning
    'unfreeze_layer': UNFREEZE_LAYER
}
# Fixed model interface configuration
MODEL_CONFIG = {
    'img_size': IMAGE_SIZE,
    'mask_size': MASK_SIZE,
    'num_classes': NUM_CLASSES
}
# Batch size processed by each replica (GPU/TPU core)
BATCH_SIZE_PER_REPLICA = 8
# Effective batch size across all synchronized replicas
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Dataset-related runtime parameters
DATASET_CONFIG = {
    'shuffle': SHUFFLE_SIZE,
    'batch_size': GLOBAL_BATCH_SIZE,
    'auto': AUTO
}
# Log final batch size for verification
print(f'Global Batch size: {GLOBAL_BATCH_SIZE}')

Global Batch size: 8


---
### 2.2 Dataset Class Mapping (Original Labels)

Then we load the original class-to-index mapping used during dataset creation.  
Preserving and explicitly logging this mapping is critical for **label integrity**, **debugging**, and **correct interpretation** of evaluation results.

In [5]:
# Load original dataset class-to-index mapping
class_mapping_path = '../../data/class_mapping.json'
with open(class_mapping_path, 'r') as f:
    class_mapping = json.load(f)

# Display mapping for validation and traceability
print(class_mapping)

{'COVID': 0, 'Normal': 1, 'Viral Pneumonia': 2, 'Lung_Opacity': 3}


### 2.5 Dataset Class Mapping (Filtered Unhealthy Classes)

Now load the remapped label definitions used after filtering out healthy samples.  
The resulting mapping aligns the dataset with the **three-class disease classification task**, ensuring consistent label semantics throughout training and evaluation.

In [6]:
# Load remapped labels after excluding healthy samples
unhealthy_class_mapping_path = '../../data/unhealthy_mapping.json'
with open(unhealthy_class_mapping_path, 'r') as f:
    unhealthy_class_mapping = json.load(f)

# Display final disease-class mapping
print(unhealthy_class_mapping)

{'COVID': 0, 'Viral Pneumonia': 1, 'Lung Opacity': 2}


---
## Section 3 – Dataset Construction for Optuna Optimization

A lightweight training and validation dataset is constructed to support efficient Optuna trials.  
Rather than using the full dataset, a **restricted subset of TFRecord files** is intentionally selected to reduce computational cost while preserving representative data statistics.

This setup enables rapid hyperparameter exploration without compromising the validity of the optimization process.

### 3.1 TFRecord File Selection for Optimization

A small subset of TFRecord files is extracted from the full dataset for Optuna-driven experimentation.  
Using a reduced training split significantly accelerates trial execution while maintaining consistent preprocessing and label semantics.

In [7]:
# Collect and sort all available TFRecord files
all_files = sorted(tf.io.gfile.glob(os.path.join(DATA_DIR, '*.tfrecord')))
# Subset selection for Optuna trials (lightweight training set)
sub_train_files = all_files[:3]
# Validation split reserved from the dataset tail
val_files = all_files[-1:]

### 3.2 Parsing, Label Remapping, and Preprocessing Configuration

Parsing logic, label remapping, ROI-based preprocessing, and backbone-specific input normalization are assembled into a unified dataset configuration.  
This ensures that all Optuna trials operate on **identical input transformations**, isolating optimization effects from data inconsistencies.

In [8]:
# TFRecord parsing function with fixed image and mask resolution
parse_fn = make_parse_fn(config=MODEL_CONFIG)
# Remap labels to a contiguous multi-class space after filtering
remap_for_multiclass = make_remap_for_multiclass(NUM_CLASSES)
# Attach dataset transformation components to the shared config
DATASET_CONFIG['parse_fn'] = parse_fn
DATASET_CONFIG['remap'] = remap_for_multiclass
DATASET_CONFIG['roi'] = lung_roi_preprocess
DATASET_CONFIG['preprocess'] = preprocess_input

### 3.3 Lightweight Data Augmentation Pipeline

A restrained augmentation pipeline is used during optimization to introduce mild spatial and intensity variability.  
Augmentations are intentionally conservative to stabilize Optuna evaluations while still improving robustness to minor appearance changes.

In [9]:
# Sequential augmentation pipeline applied at the batch level
light_augmentation = tf.keras.Sequential([
    # Geometric transformations
    tfl.RandomFlip('horizontal', input_shape= IMAGE_SIZE + (3,)),
    tfl.RandomRotation(0.05, interpolation='bilinear', fill_mode='nearest'),
    tfl.RandomZoom(0.05, interpolation='bilinear', fill_mode='nearest'),
    
    # Intensity and contrast adjustments
    tfl.RandomContrast(0.05),
    tfl.RandomBrightness(0.05)
])

/home/a1mohamad/ai-env/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 3.4 Training and Validation Dataset Assembly

Training and validation datasets are instantiated using a shared construction pipeline.  
Augmentation is applied **only to the training split**, while validation data remains deterministic for reliable metric comparison.

In [10]:
# Build training dataset for Optuna trials
sub_train_dataset = multiclass_dataset(
    sub_train_files,
    config=DATASET_CONFIG,
    is_training=True,
    image_augmentation=light_augmentation
)

# Build validation dataset without stochastic augmentation
val_dataset = multiclass_dataset(
    val_files,
    config=DATASET_CONFIG,
    is_training=False,
    image_augmentation=light_augmentation
)

### 3.5 Dataset Metadata and Optimization Parameters

Dataset statistics are computed in a single pass to derive:
- Effective training and validation steps
- Class weights for imbalanced learning
- Optuna-specific runtime parameters

These values are injected directly into the optimization configuration to keep trial logic compact and consistent.

In [11]:
# Extract dataset statistics required for optimization
optuna_metadata = get_dataset_metadata(sub_train_dataset, GLOBAL_BATCH_SIZE)
# Number of training steps per Optuna trial
optuna_steps = optuna_metadata["steps"]
# Class weights derived from dataset distribution
weights = optuna_metadata["weights"]
weights = np.array([weights[k] for k in weights.keys()])
# Validation step count
validation_steps = count_steps_from_dataset(val_dataset)
# Log derived metadata for verification
print(f"Optuna Training Steps: {optuna_steps}\nValidation Steps: {validation_steps}")
print(f"Class weights: {weights}")
# Inject dataset-dependent parameters into optimization config
OPTIMIZATION_CONFIG['optuna_steps'] = int(optuna_steps)
OPTIMIZATION_CONFIG['validation_steps'] = int(validation_steps)
OPTIMIZATION_CONFIG['class_weights'] = weights

2026-01-10 22:07:01.202135: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144


Optuna Training Steps: 411
Validation Steps: 131
Class weights: [1.0320151  2.781726   0.59825325]


## Section 4 – Optuna Optimization Objective and Study Execution

This section defines the core **Optuna optimization logic** used to identify the best training and regularization hyperparameters for the disease classification model.

The classifier architecture is **fixed and inherited from Phase 1**, while Optuna explores optimization-related parameters such as learning rates, regularization strength, dropout, and class-weight scaling.  
Model performance is evaluated using a **custom penalized $F_1$ score**, designed to balance accuracy, calibration, and generalization.


### 4.1 Loading Best Architecture from Phase 1

Architecture metadata obtained from the Phase 1 Optuna search is loaded and reused without modification.  
This guarantees a clean separation between **architecture search** and **optimization search**, ensuring that improvements are attributable solely to training dynamics.


In [12]:
# Load best architecture metadata obtained from Phase 1
best_arch_dir = './phase1_architecture/best_architecture.json'
with open(best_arch_dir, 'r') as f:
    best_arch = json.load(f)
# Log selected architecture details for traceability
print(f"Best Architecture metadata: {best_arch}")
print(
    f"Best Architecture Hyperparameters:\n"
    f"Number of Dense layers: {best_arch['best_hparams']['num_layers']}\n"
    f"Dense units in order: {best_arch['best_hparams']['dense_units']}"
)
# Attach Phase 1 hyperparameters to optimization config
OPTIMIZATION_CONFIG['phase1_hparams'] = best_arch['best_hparams']

Best Architecture metadata: {'best_trial_number': 16, 'best_value': 0.924062991142273, 'best_hparams': {'num_layers': 2, 'dense_units': [256, 64]}, 'phase1_settings': {'loss': 'CategoricalCrossentropy', 'optimizer': 'Adam', 'lr': 0.0003, 'weight_decay': 0, 'dropout_rate': 0.1}, 'timestamp': 1767628988.7223353, 'seed': 28}
Best Architecture Hyperparameters:
Number of Dense layers: 2
Dense units in order: [256, 64]


### 4.2 Optimization Hyperparameter Search Space Definition

The Optuna search space is defined for **optimization-specific hyperparameters** while preserving the Phase 1 architecture.  
These parameters control learning dynamics, regularization strength, and class imbalance handling during training.

In [14]:
def optimization_hparams(trial, hparams=best_arch["best_hparams"]):
    """
    Define the Optuna search space for optimization-related hyperparameters.

    Architecture parameters are reused from Phase 1, while learning rates,
    regularization, and class-weight scaling are tuned in this phase.
    """
    num_layers = hparams["num_layers"]

    # Stage-wise learning rates
    hparams["lr_stage1"] = trial.suggest_float("lr_stage1", 1e-4, 1e-3, log=True)
    hparams["lr_stage2"] = trial.suggest_float("lr_stage2", 5e-6, 5e-5, log=True)

    # Regularization parameters
    hparams["weight_decay"] = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hparams["label_smoothing"] = trial.suggest_categorical(
        "label_smoothing", [0.0, 0.025, 0.05, 0.075, 0.1]
    )

    # Dropout applied to classifier head
    hparams["dropout_rate"] = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.05)

    # Power applied to class weights (0 → uniform, 1 → raw imbalance)
    hparams["weight_power"] = trial.suggest_float("weight_power", 0.0, 1.0)
    
    return hparams

### 4.3 Optuna Objective Function with Two-Stage Training

The objective function defines a **two-stage training protocol**:

1. **Warm-up stage** with a frozen backbone  
2. **Fine-tuning stage** with selective backbone unfreezing and cosine learning-rate decay  

Each trial is evaluated using a penalized $F_1$ score that accounts for:
- Precision–recall imbalance
- Overfitting via loss divergence

In [15]:
def objective_optimization(trial, config=OPTIMIZATION_CONFIG):
    """
    Optuna objective function implementing a two-stage training strategy
    with frozen and unfrozen backbone phases.
    """
    print('=' * 180)
    print(f"Trial {trial.number + 1}/{config['opt_trials']} started...")

    model = None
    history = None
    callbacks = None

    # Training schedule parameters
    warmup_epoch = config['opt_warmup_epoch']
    unfreeze_epoch = config["opt_unfreeze_epoch"]
    total_epoch = warmup_epoch + unfreeze_epoch

    # Dataset and optimization metadata
    optuna_steps = config['optuna_steps']
    validation_steps = config['validation_steps']
    raw_class_weights = config['class_weights']
    arch_hparams = config['phase1_hparams']
    unfreeze_layer = config["unfreeze_layer"]

    # Sample optimization hyperparameters
    hparams = optimization_hparams(trial, hparams=arch_hparams)
    trial.set_user_attr("hparams", hparams)

    dropout_rate = hparams["dropout_rate"]
    p = hparams["weight_power"]

    # Apply power scaling to class weights
    class_weights = dict(enumerate(np.power(raw_class_weights, p)))

    # Loss function with label smoothing
    loss = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=hparams['label_smoothing']
    )

    try:
        # ---------------------------
        # Stage 1: Frozen backbone
        # ---------------------------
        with strategy.scope():
            optimizer = tf.keras.optimizers.AdamW(
                learning_rate=hparams["lr_stage1"],
                weight_decay=hparams["weight_decay"]
            )

            model = densenet_model(
                hparams,
                dropout_rate=dropout_rate,
                config=MODEL_CONFIG,
                phase='opt'
            )
            model = compile_model(model, loss=loss, optimizer=optimizer)

        history = model.fit(
            sub_train_dataset.repeat(),
            validation_data=val_dataset.repeat(),
            epochs=warmup_epoch,
            steps_per_epoch=optuna_steps,
            validation_steps=validation_steps,
            class_weight=class_weights
        )

        # ---------------------------
        # Stage 2: Fine-tuning
        # ---------------------------
        with strategy.scope():
            model = unfreeze_backbone(
                model,
                backbone_name='densenet',
                unfreeze_layer=unfreeze_layer
            )

            decay_steps = total_epoch * optuna_steps
            lr = tf.keras.optimizers.schedules.CosineDecay(
                decay_steps=decay_steps,
                initial_learning_rate=hparams["lr_stage2"],
                alpha=0.0
            )

            optimizer = tf.keras.optimizers.AdamW(
                learning_rate=lr,
                weight_decay=hparams["weight_decay"]
            )
            model = compile_model(model, loss=loss, optimizer=optimizer)

        earlystop_cb = EarlyStopping(
            monitor='val_f1_score',
            patience=3,
        )
        callbacks = [earlystop_cb]

        history = model.fit(
            sub_train_dataset.repeat(),
            validation_data=val_dataset.repeat(),
            initial_epoch=warmup_epoch,
            epochs=total_epoch,
            steps_per_epoch=optuna_steps,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=class_weights
        )

        # Compute penalized F1 score
        score, best_f1, best_prec, best_rec = penalized_f1_score(
            history,
            config=PENALIZED_F1_CONFIG,
            mode='mean',
            loss=True
        )

        # Log final losses for diagnostics
        last_train_loss = history.history["loss"][-1]
        last_val_loss = history.history["val_loss"][-1]

        trial.set_user_attr("phase2_score", float(score))
        print(
            f"Penalized F1: {score:.4f}, "
            f"Best F1: {best_f1:.4f}, "
            f"P: {best_prec:.4f}, "
            f"R: {best_rec:.4f}"
        )
        print(f"Last Epoch --> Train Loss: {last_train_loss} | Val Loss: {last_val_loss}")

        return score

    finally:
        # Ensure full cleanup between trials
        cleanup(model, history, callbacks)


### 4.4 Optuna Study Creation and Execution

An Optuna study is created using **SQLite-based persistence**, allowing trials to be resumed safely across sessions.  
The study maximizes the penalized $F_1$ score over a fixed number of trials.


In [ ]:
# Persistent storage for Optuna study
storage_dir = "sqlite:///phase2_optimization/diseases-phase2_optimization.db"

study = optuna.create_study(
    direction='maximize',
    storage=storage_dir,
    load_if_exists=True
)

# Launch optimization process
study.optimize(
    lambda trial: objective_optimization(trial, config=OPTIMIZATION_CONFIG),
    n_trials=OPTIMIZATION_CONFIG['opt_trials'],
    gc_after_trial=True
)

# Extract best trial information
best_trial = study.best_trial
best_hparams = best_trial.user_attrs.get("hparams", None)

### 4.5 Saving Phase 2 Optimization Metadata

Final optimization results and experimental settings are serialized for reproducibility, reporting, and downstream training stages.

In [30]:
# Collect metadata for long-term storage
metadata = { 
    "best_trial_number": int(best_trial.number), 
    "best_value": float(best_trial.value), 
    "best_hparams": best_hparams, 
    "phase2_settings": { 
        "loss": "CategoricalCrossentropy", 
        "optimizer": "AdamW", 
        "lr_schedules": "CosineDecay", 
        "warmup_epochs": OPTIMIZATION_CONFIG["opt_warmup_epoch"],
        "unfreeze_epochs": OPTIMIZATION_CONFIG["opt_unfreeze_epoch"],
        "unfreeze_layer": OPTIMIZATION_CONFIG["unfreeze_layer"],
        "trials": OPTIMIZATION_CONFIG["opt_trials"]
    }, 
    "timestamp": time.time(), 
    "seed": SEED
}

In [32]:
# Persist metadata to disk
metadata_dir = "./phase2_optimization/diseases-best_hparams.json"
with open(metadata_dir, "w") as f:
    json.dump(metadata, f, indent=2)

## **Conclusion & Next Steps**

### **Conclusion**

In this notebook, we successfully conducted **Phase 2: Optimization Hyperparameters Search** for a DenseNet-based head classifier using **Optuna**.  

Key outcomes include:

- Efficient exploration of **Optimization hyperparameters** (two stages learning rates, weight decay and etc.). 
- Implementation of a **DenseNet121 backbone** frozen with pre-trained weights in stage 1, then unfrozen from `conv5_block1_0_bn` layer in stage 2.  
- Use of **ROI-based preprocessing** to focus on lung regions while preserving DenseNet’s expected input format.  
- Creation of **training and validation Subset of datasets** optimized for Optuna trials, with class re-mapping and also search for best `class weight power`.  
- Definition of a **custom objective function** using **penalized F1 score** to account for class imbalance and stabilize metric evaluation with loss distance penalized and also gap between `Precision` and `Recall`.  
- Establishment of a **persistent Optuna study** for reproducibility and easy continuation of hyperparameter search.  

This notebook lays the foundation for **reliable, reproducible, and professional hyperparameter optimization**, which is essential in medical AI applications where **accuracy, precision, and recall** are critical.

---

### **Next Steps**

The next phase of this project will focus on:

1. **Fine-tuning & Evaluation**  
   - Conduct full model training with **frozen and partially unfrozen DenseNet layers**.  
   - Evaluate on **independent test sets** to ensure generalization.   
2. **Developing Step**
    - Gather all models and go one use these models in Real World!

Now we ready for **Training DenseNet121 diseases classification Model**.


## **Resources & References**

#### Official Documentation

**TensorFlow & DenseNet**

- [TensorFlow main site](https://www.tensorflow.org) — comprehensive docs, tutorials, API guides  
- [DenseNet121 in TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/applications/DenseNet121) — TF Keras application reference  
- [Keras Applications — DenseNet](https://keras.io/api/applications/densenet/) — overview and usage examples  

**Optuna (Hyperparameter Optimization)**

- [Optuna Official Site](https://optuna.org) — documentation, examples, guides  
- [Optuna Tutorials & API Docs](https://optuna.github.io/optuna-web-dev/) — how to define objectives, create studies, and use pruners  
- [Optuna GitHub Repository](https://github.com/optuna/optuna) — source code, issues, and examples  

---

#### Medical Imaging with DenseNet (Selected Papers)

- [A comparative study of CNNs for COVID‑19 detection on chest X‑rays](https://link.springer.com/article/10.1186/s13634-021-00755-1) — DenseNet121 performance vs other backbones  
- [Evaluation of CNNs for COVID‑19 classification on chest X‑rays](https://arxiv.org/abs/2109.02415) — DenseNet and other CNNs evaluated for COVID‑19  
- [Automated pneumonia detection using DenseNet121 and other CNNs](https://academic.oup.com/bjr/article-abstract/doi/10.1259/bjr.20201263/7476744) — DenseNet121 effectiveness for pneumonia vs normal classification  

---

#### Additional Reading

- [Optuna Framework Paper (KDD 2019)](https://optuna.github.io/optuna-web-dev/) — Takuya Akiba et al., formal research paper describing Optuna  

---

*These resources provide both practical implementation guidance and scientific context for DenseNet-based lung disease classification and Optuna hyperparameter optimization.*
